# Treinamento com SageMaker Training Jobs

Este notebook demonstra como treinar modelos usando **SageMaker Training Jobs** com algoritmos **built-in**.

## Diferença: Treino Local vs Treino Gerenciado

| Aspecto | Treino Local | Treino Gerenciado (SageMaker) |
|---------|--------------|-------------------------------|
| **Onde executa** | Na sua máquina/notebook | Em instâncias dedicadas da AWS |
| **Escala** | Limitado aos recursos locais | Escala para qualquer tamanho |
| **Custo** | Paga pelo tempo do notebook | Paga apenas pelo tempo de treino |
| **Gerenciamento** | Você gerencia tudo | AWS gerencia a infraestrutura |

---

## 1. Configuração Inicial

Primeiro, importamos as bibliotecas necessárias e configuramos o ambiente SageMaker.

In [ ]:
# Bibliotecas básicas
import pandas as pd
import numpy as np
import boto3

# Bibliotecas SageMaker
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

# Para preparar os dados
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

print(f"SageMaker SDK version: {sagemaker.__version__}")

### 1.1 Configurar Sessão SageMaker

A sessão SageMaker gerencia a comunicação com os serviços AWS.

In [ ]:
# Criar sessão SageMaker
sagemaker_session = sagemaker.Session()

# Obter a região atual
region = sagemaker_session.boto_region_name

# Obter o bucket padrão do SageMaker (criado automaticamente)
bucket = sagemaker_session.default_bucket()

# Obter a role de execução do SageMaker
role = sagemaker.get_execution_role()

print(f"Região: {region}")
print(f"Bucket S3: {bucket}")
print(f"Role: {role[:50]}...")

---

## 2. Preparar os Dados

Vamos usar o dataset **California Housing** para prever valores de imóveis.

### 2.1 Carregar o Dataset

In [ ]:
# Carregar dataset California Housing
california = fetch_california_housing()

# Criar DataFrame
X = pd.DataFrame(california.data, columns=california.feature_names)
y = pd.Series(california.target, name='MedHouseVal')

print(f"Total de registros: {len(X)}")
print(f"Colunas: {list(X.columns)}")
X.head()

### 2.2 Dividir em Treino e Validação

In [ ]:
# Dividir dados: 80% treino, 20% validação
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Dados de treino: {len(X_train)} registros")
print(f"Dados de validação: {len(X_val)} registros")

### 2.3 Formatar Dados para o SageMaker XGBoost

O XGBoost built-in do SageMaker espera os dados em formato **CSV sem cabeçalho**, com a **coluna target na primeira posição**.

In [ ]:
# Criar DataFrames com target na primeira coluna
train_data = pd.concat([y_train.reset_index(drop=True), 
                        X_train.reset_index(drop=True)], axis=1)

val_data = pd.concat([y_val.reset_index(drop=True), 
                      X_val.reset_index(drop=True)], axis=1)

# Salvar como CSV (sem cabeçalho e sem índice)
train_data.to_csv('train.csv', index=False, header=False)
val_data.to_csv('validation.csv', index=False, header=False)

print("Arquivos criados:")
print(f"  - train.csv ({len(train_data)} linhas)")
print(f"  - validation.csv ({len(val_data)} linhas)")

### 2.4 Fazer Upload para o S3

O SageMaker Training Job lê os dados diretamente do S3.

In [ ]:
# Definir prefixo (pasta) no S3
prefix = 'treinamento-xgboost'

# Upload dos arquivos para S3
train_s3_path = sagemaker_session.upload_data(
    path='train.csv', 
    bucket=bucket, 
    key_prefix=f'{prefix}/train'
)

val_s3_path = sagemaker_session.upload_data(
    path='validation.csv', 
    bucket=bucket, 
    key_prefix=f'{prefix}/validation'
)

print("Dados enviados para o S3:")
print(f"  Treino: {train_s3_path}")
print(f"  Validação: {val_s3_path}")

---

## 3. Configurar o Training Job

Agora vamos configurar o Training Job usando o **XGBoost built-in** do SageMaker.

### 3.1 Obter a Imagem do XGBoost Built-in

O SageMaker fornece imagens Docker prontas com algoritmos built-in.

In [ ]:
# Obter a URI da imagem XGBoost para nossa região
xgboost_image = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.7-1"  # Versão do XGBoost
)

print(f"Imagem XGBoost: {xgboost_image}")

### 3.2 Definir Hiperparâmetros

Os hiperparâmetros controlam como o modelo aprende:

| Parâmetro | Descrição |
|-----------|----------|
| `num_round` | Número de árvores (iterações) |
| `max_depth` | Profundidade máxima das árvores |
| `eta` | Taxa de aprendizado |
| `objective` | Tipo de problema (regressão) |

In [ ]:
# Definir hiperparâmetros do XGBoost
hyperparameters = {
    "num_round": "100",           # Número de árvores
    "max_depth": "5",             # Profundidade máxima
    "eta": "0.2",                 # Taxa de aprendizado
    "objective": "reg:squarederror",  # Problema de regressão
    "subsample": "0.8",           # Fração de dados por árvore
    "colsample_bytree": "0.8"     # Fração de features por árvore
}

print("Hiperparâmetros configurados:")
for k, v in hyperparameters.items():
    print(f"  {k}: {v}")

### 3.3 Criar o Estimator

O Estimator define a configuração completa do Training Job.

In [ ]:
# Definir onde salvar o modelo treinado
output_path = f's3://{bucket}/{prefix}/output'

# Criar o Estimator
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=xgboost_image,           # Imagem do XGBoost
    role=role,                          # Role de execução
    instance_count=1,                   # Quantidade de máquinas
    instance_type='ml.m5.large',        # Tipo de instância
    volume_size=5,                      # Disco em GB
    output_path=output_path,            # Onde salvar o modelo
    hyperparameters=hyperparameters,    # Hiperparâmetros
    sagemaker_session=sagemaker_session
)

print("Estimator criado com sucesso!")
print(f"Instância: ml.m5.large")
print(f"Output: {output_path}")

### 3.4 Configurar os Canais de Dados

Os canais definem de onde o SageMaker vai ler os dados.

In [ ]:
# Configurar canais de entrada
train_input = TrainingInput(
    s3_data=train_s3_path,
    content_type='csv'  # Formato dos dados
)

val_input = TrainingInput(
    s3_data=val_s3_path,
    content_type='csv'
)

print("Canais de dados configurados:")
print(f"  train: {train_s3_path}")
print(f"  validation: {val_s3_path}")

---

## 4. Executar o Training Job

Agora vamos iniciar o treinamento. O SageMaker irá:
1. Provisionar uma instância ml.m5.large
2. Baixar os dados do S3
3. Executar o treinamento XGBoost
4. Salvar o modelo no S3
5. Desligar a instância

In [ ]:
# Iniciar o Training Job
print("Iniciando Training Job...")
print("(Isso pode levar alguns minutos)\n")

xgb_estimator.fit({
    'train': train_input,
    'validation': val_input
})

print("\nTreinamento concluído!")

---

## 5. Verificar Resultados

Após o treinamento, podemos verificar onde o modelo foi salvo e as métricas.

In [ ]:
# Verificar onde o modelo foi salvo
model_artifacts = xgb_estimator.model_data

print("Resultados do Training Job:")
print(f"  Job name: {xgb_estimator.latest_training_job.name}")
print(f"  Modelo salvo em: {model_artifacts}")

### 5.1 Verificar Métricas do Treinamento

O SageMaker registra métricas automaticamente no CloudWatch.

In [ ]:
# Obter métricas do training job
from sagemaker.analytics import TrainingJobAnalytics

job_name = xgb_estimator.latest_training_job.name

# Buscar métricas
metrics = TrainingJobAnalytics(
    training_job_name=job_name,
    metric_names=['validation:rmse', 'train:rmse']
)

# Exibir métricas
metrics_df = metrics.dataframe()
if not metrics_df.empty:
    print("Métricas do treinamento:")
    print(metrics_df.tail(10))
else:
    print("Métricas ainda não disponíveis (aguarde alguns segundos)")

---

## 6. Limpeza (Opcional)

Remover arquivos locais temporários.

In [ ]:
# Remover arquivos locais
import os

for file in ['train.csv', 'validation.csv']:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removido: {file}")

print("\nLimpeza concluída!")

---

## Resumo

Neste notebook, aprendemos a:

1. **Configurar** o ambiente SageMaker (sessão, bucket, role)
2. **Preparar** os dados no formato esperado pelo XGBoost
3. **Enviar** os dados para o S3
4. **Configurar** o Training Job com hiperparâmetros
5. **Executar** o treinamento gerenciado
6. **Verificar** os resultados e métricas

### Fluxo do Training Job

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  Preparar   │    │   Upload    │    │  Training   │    │   Modelo    │
│   Dados     │───▶│    S3       │───▶│    Job      │───▶│  Salvo S3   │
│  (local)    │    │             │    │ (instância) │    │             │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
```

### Próximos Passos

- Fazer deploy do modelo como endpoint
- Testar previsões em tempo real
- Explorar hyperparameter tuning automático